In [1]:
config = {
    "base_dir":        "/storage/vbsjjlnu/VBSjjlnu_v7/",
    "plot_config":     "FullRun2_v7",
    "cut":             "boost_sig",
    "model_version":   "v4_b",
}


import os

config_base_dir = os.path.join(config["base_dir"], config["plot_config"])

# create the model directory
model_dir   = os.path.join(config_base_dir, config["cut"] , "models",  config["model_version"])
os.makedirs(model_dir, exist_ok=True)

import yaml
model_config_file = open(model_dir + "/model_config.yml", "r")
model_config = yaml.safe_load(model_config_file)

for key in ["samples_version", "cols"]:
    config[key] = model_config[key]

In [2]:
import tensorflow as tf


model = tf.keras.models.load_model(os.path.join(model_dir, "model.h5"))


In [3]:
tf.keras.models.save_model(model, model_dir+"/model_output", include_optimizer=False )

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /storage/vbsjjlnu/VBSjjlnu_v7/FullRun2_v7/boost_sig/models/v4_b/model_output/assets


In [4]:
model.save(os.path.join(model_dir, "saved_model"))

INFO:tensorflow:Assets written to: /storage/vbsjjlnu/VBSjjlnu_v7/FullRun2_v7/boost_sig/models/v6_b/saved_model/assets


In [5]:
import tensorflow as tf


# for tf v2, fgo into v1 compatibility mode
if tf.__version__.startswith("2."):
    tf = tf.compat.v1
tf.disable_eager_execution()

sess = tf.Session()

tf.keras.backend.set_session(sess)


model = tf.keras.models.load_model(os.path.join(model_dir, "model.h5"))


#————- Salvo qui ————
# inputs:  ['Input']
print('inputs: ', [input.op.name for input in model.inputs])

# outputs:  ['Output/Sigmoid']
print('outputs: ', [output.op.name for output in model.outputs])

outputs = [output.op.name for output in model.outputs]
constant_graph = tf.graph_util.convert_variables_to_constants(
    sess, sess.graph.as_graph_def(), outputs)
tf.train.write_graph(constant_graph, model_dir, "model.pb", as_text=False)

## save tensorflow metadata
with open(os.path.join(model_dir, "tf_metadata.txt"), "w") as f:
    f.write(str(model.inputs[0].name) + " " + str(model.outputs[0].name) + "\n")



inputs:  ['dense_77_input']
outputs:  ['dense_82/Sigmoid']
INFO:tensorflow:Froze 32 variables.
INFO:tensorflow:Converted 32 variables to const ops.


In [6]:

import os
import pickle

import yaml
yaml_vars = yaml.safe_load(open(os.path.join(model_dir, "variables.yml"), "r"))

scaler = pickle.load(open(os.path.join(model_dir, "scaler_model.pkl"), 'rb'))
with open(os.path.join(model_dir, "output_scaler.txt"), "w") as f:
    for var, mean, scale in zip(yaml_vars, scaler.mean_, scaler.scale_):
        f.write(var + " " + str(mean) + " " + str(scale) + "\n")


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator StandardScaler from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [3]:
import tensorflow as tf

# for tf v2, fgo into v1 compatibility mode
if tf.__version__.startswith("2."):
    tf = tf.compat.v1
    
tf.compat.v1.disable_eager_execution()

sess = tf.compat.v1.Session()

tf.compat.v1.keras.backend.set_session(sess)


model = tf.compat.v1.keras.models.load_model(os.path.join(model_dir, "model.h5"))


# save at as a constant graph
outputs = [...]
gd = sess.graph.as_graph_def()
constant_graph = tf.compat.v1.graph_util.convert_variables_to_constants(
    sess, gd , outputs)
#tf.train.write_graph(constant_graph, "/path/to", "constantgraph.pb", as_text=False)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


AssertionError: Ellipsis is not in graph